In [ ]:
%pip install pymupdf pypdf langchain_community

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('data/2040_seoul_plan.pdf')
data_seoul = loader.load()
print(data_seoul)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(data_seoul)

for i, split in enumerate(all_splits):
    print(f'split {i+1}: -----------------')
    print(split)

In [ ]:
print(type(all_splits[0]))

In [ ]:
print(all_splits[50].page_content)
print(': -----------------: -----------------: -----------------')
print(all_splits[51].page_content)

In [ ]:
%pip install langchain_chroma

In [ ]:
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
import os
load_dotenv()

OPEN_AI_KEY = os.getenv('DEV_OPENAI_API_KEY')
embedding = OpenAIEmbeddings(model='text-embedding-3-large',api_key=OPEN_AI_KEY)
v = embedding.embed_query("뉴욕의 온실가스 저감 정책은 뭐야")
print(v)
print(len(v))

In [ ]:
from langchain_chroma import Chroma
import os 

persis_directory = './chroma_store'

if not os.path.exists(persis_directory):
    print("크로마 스토어 생성")
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embedding,
        persist_directory=persis_directory,
    )
else :
    print("load existing chroma store")
    vectorstore = Chroma(
        embedding_function=embedding,
        persist_directory=persis_directory,
    )

In [ ]:
retriever = vectorstore.as_retriever(k=3)
docs = retriever.invoke("서울시의 환경 정책이 궁금해")

for d in docs:
    print(d, '\n\n')
    

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents